In [1]:
import sqlite3
import nltk
from simhash import Simhash
import numpy as np

In [15]:
db = sqlite3.connect('../../data/MSN_technology.db')
cursor = db.cursor()
info_all = cursor.execute("SELECT * FROM MSN_TECH")
articles = [info[5] for info in info_all]

In [3]:
def preprocess_data(articles):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    all_tokens = []
    for i, article in enumerate(articles):
        tokens = ' '.join([token for token in nltk.word_tokenize(article) \
                    if token not in stopwords])
        all_tokens.append(tokens)
    return all_tokens

In [4]:
tokens = preprocess_data(articles)

In [5]:
codes = []
for i in range(len(tokens)):
    codes.append(Simhash(tokens[i]))

In [6]:
s = []
for i in range(len(tokens)):
    s.append([])
for i in range(len(tokens) - 1):
    code1 = codes[i]
    for j in range(i + 1, len(tokens)):
        code2 = codes[j]
        dist = code1.distance(code2)
        if dist < 16:
            s[i].append(j)
            s[j].append(i)

In [7]:
best8 = np.zeros((len(s), 8), dtype=np.int32)
best8 -= 1
for i in range(len(s)):
    g = sorted([(codes[i].distance(codes[k]), k) for k in s[i]])
    for j in range(min(8, len(g))):
        best8[i][j] = g[j][1]

np.save('related.npy', best8)

In [14]:
print(list(best8[7359]))

[547, 7027, 6362, 3023, -1, -1, -1, -1]


In [17]:
for i in range(7360):
    cursor.execute('UPDATE MSN_TECH SET related = ? WHERE _id = ?', (str(list(best8[i])), i))

In [18]:
db.commit()

In [19]:
db.close()

In [20]:
best8[7140]

array([-1, -1, -1, -1, -1, -1, -1, -1])